In [1]:
import networkx as nx
import pandas as pd
from node2vec import Node2Vec
import node2vec

In [2]:
from create_graph import graph
from node2vec_embed import node_2vec



In [3]:
himgraph = graph()

In [4]:
df_features = pd.read_csv('/home/lxu/Downloads/transformed-features.tsv',sep ='\t',low_memory=False)
df_features.head()

,compound_id,compound_name,disease_id,disease_name,category,status,prior_prob,prior_logit,degree_CbG,degree_CcSE,...,dwpc_CuGaDuGaD,dwpc_CuGcG<rGaD,dwpc_CuGcGiGaD,dwpc_CuGdDpSpD,dwpc_CuGr>GbCtD,dwpc_CuGuCrCtD,dwpc_CuGuCtD,dwpc_CuGuCtDrD,dwpc_CuGuCuGaD,dwpc_CuGuDuGaD
0,DB01048,Abacavir,DOID:10652,Alzheimer's disease,NaN,0,0.004753,-5.344,-0.2955,0.8146,...,-0.5028,-0.4211,-0.45810,-0.53990,-0.357,-0.3849,-0.375,-0.5642,-0.5555,-0.4952
1,DB05812,Abiraterone,DOID:10652,Alzheimer's disease,NaN,0,0.004753,-5.344,0.5495,0.5554,...,-0.5028,-0.4211,-0.45810,-0.53990,-0.357,-0.3849,-0.375,-0.5642,-0.5555,-0.4952
2,DB00659,Acamprosate,DOID:10652,Alzheimer's disease,NaN,0,0.004753,-5.344,0.9053,1.0220,...,-0.5028,-0.4211,-0.45810,-0.53990,-0.357,-0.3849,-0.375,-0.5642,-0.5555,-0.4952
3,DB00284,Acarbose,DOID:10652,Alzheimer's disease,NaN,0,0.004753,-5.344,0.4328,0.3460,...,0.4304,0.6755,-0.07508,-0.08597,-0.357,-0.3849,-0.375,-0.5642,0.8816,0.6339
4,DB01193,Acebutolol,DOID:10652,Alzheimer's disease,NaN,0,0.004753,-5.344,0.1403,0.7966,...,-0.5028,-0.4211,-0.45810,-0.53990,-0.357,-0.3849,-0.375,-0.5642,-0.5555,-0.4952


In [5]:
df_positive = df_features.loc[df_features['status']== 1][0:]
len(df_positive)

20

In [ ]:
df_negative = df_features.loc[df_features['status']== 0][0:40]
df_negative[1]

In [9]:
subgraph_nodeslist=[]
for index,row in df_positive.iterrows():
    for path in nx.all_simple_paths(himgraph, source='Compound::'+row['compound_id'], target='Disease::'+row['disease_id'],cutoff=3):
        #print(path)
        subgraph_nodeslist=subgraph_nodeslist+path
for i,r in df_negative.iterrows():
     for path in nx.all_simple_paths(himgraph, source='Compound::'+r['compound_id'], target='Disease::'+r['disease_id'],cutoff=3):
        #print(path)
        subgraph_nodeslist=subgraph_nodeslist+path   
subgraph_nodes_list = list(set(subgraph_nodeslist))
subgraph_nodes_list


['Side Effect::C0011603',
 'Side Effect::C0033975',
 'Gene::7105',
 'Side Effect::C0026266',
 'Gene::9871',
 'Compound::DB00433',
 'Side Effect::C0233514',
 'Gene::8844',
 'Side Effect::C0017150',
 'Compound::DB00635',
 'Gene::9500',
 'Side Effect::C0025222',
 'Side Effect::C0235490',
 'Gene::1389',
 'Gene::440',
 'Side Effect::C0232493',
 'Side Effect::C0013595',
 'Gene::4133',
 'Side Effect::C0001349',
 'Side Effect::C0034186',
 'Gene::3593',
 'Gene::5319',
 'Gene::145264',
 'Compound::DB01078',
 'Side Effect::C1384353',
 'Compound::DB00455',
 'Gene::5371',
 'Gene::24149',
 'Side Effect::C0038661',
 'Gene::10158',
 'Gene::64795',
 'Gene::678',
 'Gene::7465',
 'Side Effect::C0009782',
 'Disease::DOID:184',
 'Gene::9162',
 'Side Effect::C0007682',
 'Gene::60481',
 'Gene::9782',
 'Gene::51203',
 'Side Effect::C0004936',
 'Gene::7027',
 'Gene::84722',
 'Side Effect::C0234131',
 'Compound::DB00631',
 'Compound::DB00195',
 'Side Effect::C0085584',
 'Side Effect::C0025061',
 'Compound::DB01

In [10]:
import simplejson
f = open('subgraph_nodes', 'w')
simplejson.dump(subgraph_nodes_list, f)
f.close()

In [11]:
test_graph = himgraph.subgraph(subgraph_nodes_list)
test_graph

In [12]:
positive_list=[]
positve_label=[1]*20
for _,row in df_positive[['compound_id','disease_id']].iterrows():
    positive_list.append(['Compound::'+row['compound_id'],'Disease::'+row['disease_id']])
negative_list=[]
negative_label=[0]*40
for _,row in df_negative[['compound_id','disease_id']].iterrows():
    negative_list.append(['Compound::'+row['compound_id'],'Disease::'+row['disease_id']])

In [13]:
ff= open('positive_list', 'w')
simplejson.dump(positive_list, ff)
ff.close()

In [14]:
fff = open('negative_list','w')
simplejson.dump(negative_list, fff)
fff.close()

In [15]:
model= node_2vec(test_graph)


Computing transition probabilities: 100%|██████████| 3422/3422 [13:15<00:00,  3.19it/s]  


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {SIGKILL(-9)}

In [ ]:
pair_list=positive_list
edge_vecs = []
edges_embs = node2vec.edges.HadamardEmbedder(keyed_vectors=model.wv)
for i in pair_list:
        node1=i[0]
        node2=i[1]
        vec= edges_embs[(node1,node2)].tolist()
        edge_vecs.append(vec)
print(edge_vecs)

In [ ]:
npair_list=negative_list
nedge_vecs = []
edges_embs = node2vec.edges.HadamardEmbedder(keyed_vectors=model.wv)
for i in npair_list:
        node1=i[0]
        node2=i[1]
        vec= edges_embs[(node1,node2)].tolist()
        nedge_vecs.append(vec)


In [ ]:
import numpy as np
train_x= edge_vecs[0:15]+nedge_vecs[0:30]


train_y= positve_label[0:15]+negative_label[0:30]

test_x= edge_vecs[15:]+nedge_vecs[30:]
test_y= positve_label[15:]+negative_label[30:]

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0,solver='lbfgs',
tol=1e-8).fit(train_x,train_y)

In [ ]:
clf.score(test_x,test_y)

In [ ]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(test_y, clf.predict_proba(test_x)[:,1])
roc

# Test pairs.py

In [1]:
import pairs
a,b = pairs.test_pairs('/home/lxu/Desktop/learn-master/validate/validation-statuses.tsv')
print(a[1])
print(b[1])

['Compound::DB01048', 'Disease::DOID:9206']
0


In [3]:
c,d = pairs.train_pairs('/home/lxu/Desktop/pipeline/original_graph_data/transformed-features.tsv')
print(c[1])
print(d[1])

['Compound::DB05812', 'Disease::DOID:10652']
0


# Test subgraph.py

In [7]:
import subgraph
import create_graph
graph = create_graph.graph('/home/lxu/Desktop/pipeline/original_graph_data/nodes.tsv','/home/lxu/Desktop/pipeline/original_graph_data/edges.sif')
graph,positive_list,positive_label,negative_list,negative_label = subgraph.subgraph('/home/lxu/Desktop/pipeline/original_graph_data/transformed-features.tsv',graph,3,2,4)

In [8]:
import node2vec
def nodetovec (graph):
    n_model = node2vec.Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=4)
    model = n_model.fit(window=10, min_count=1, batch_words=4)
    return model
def HadamardEmbedder(model,pair_list):
    edge_vecs = []
    edges_embs = node2vec.edges.HadamardEmbedder(keyed_vectors=model.wv)
    for i in pair_list:
        node1=i[0]
        node2=i[1]
        vec= edges_embs[(node1,node2)].tolist()
        edge_vecs.append(vec)
    return edge_vecs
def AverageEmbedder(model,pair_list):
    edge_vecs = []
    edges_embs = node2vec.edges.AverageEmbedder(keyed_vectors=model.wv)
    for i in pair_list:
        node1=i[0]
        node2=i[1]
        vec= edges_embs[(node1,node2)].tolist()
        edge_vecs.append(vec)
    return edge_vecs 
def WeightedL1Embedder(model,pair_list):
    edge_vecs = []
    edges_embs = node2vec.edges.WeightedL1Embedder(keyed_vectors=model.wv)
    for i in pair_list:
        node1=i[0]
        node2=i[1]
        vec= edges_embs[(node1,node2)].tolist()
        edge_vecs.append(vec)
    return edge_vecs 
def WeightedL2Embedder(model,pair_list):
    edge_vecs = []
    edges_embs = node2vec.edges.WeightedL2Embedder(keyed_vectors=model.wv)
    for i in pair_list:
        node1=i[0]
        node2=i[1]
        vec= edges_embs[(node1,node2)].tolist()
        edge_vecs.append(vec)
    return edge_vecs  
model = nodetovec(graph)

Computing transition probabilities: 100%|██████████| 915/915 [00:24<00:00, 31.53it/s]


In [4]:
edge_vecs = HadamardEmbedder(model,positive_list)
edge_vecs

[[1.7809802293777466,
  -0.43522530794143677,
  -0.40023279190063477,
  -1.541100263595581,
  -2.33532977104187,
  -7.49254035949707,
  -6.379592418670654,
  2.3548343181610107,
  0.13776715099811554,
  0.9960280060768127,
  -0.47252947092056274,
  -5.365533351898193,
  -1.558414101600647,
  -3.986579656600952,
  -14.418861389160156,
  -4.244260311126709,
  0.3063437044620514,
  -0.6104075312614441,
  -3.727428913116455,
  0.03428065404295921,
  0.29965364933013916,
  -1.7805747985839844,
  -1.6012474298477173,
  -0.7390459179878235,
  -1.144492268562317,
  -0.04701228067278862,
  -1.9439969062805176,
  -4.4315314292907715,
  -0.13754937052726746,
  0.2707919180393219,
  -0.4913454055786133,
  -0.6228678226470947,
  1.901001214981079,
  -2.623382091522217,
  -1.0507233142852783,
  -5.290874004364014,
  0.561759352684021,
  -3.0520071983337402,
  0.1523778885602951,
  -0.4374241828918457,
  -5.342694282531738,
  -0.15067988634109497,
  -0.01646955870091915,
  -6.671309947967529,
  -4.20

In [4]:
import nodetovec
model = nodetovec.nodetovec(graph)

Computing transition probabilities: 100%|██████████| 915/915 [00:25<00:00, 36.23it/s]


In [6]:
edge_train_p = nodetovec.HadamardEmbedder(model,positive_list)[0:1]
edge_train_n = nodetovec.HadamardEmbedder(model,negative_list)[0:2]
edge_train = edge_train_p+edge_train_n
train_label= [1,0,0]
edge_test_p = nodetovec.HadamardEmbedder(model,positive_list)[1:]
edge_test_n =nodetovec.HadamardEmbedder(model,negative_list)[2:]
edge_test = edge_test_p + edge_test_n
test_label= [1,0,0]

# Test train.py

In [ ]:
import train
lg = train.train(edge_train,train_label)
roc = train.validate(lg,edge_test,test_label)